# RO47019: Intelligent Control Systems Practical Assignment
* Period: 2022-2023, Q3
* Course homepage: https://brightspace.tudelft.nl/d2l/home/500969
* Instructor: Cosimo Della Santina (C.DellaSantina@tudelft.nl)
* Teaching assistant: Ruben Martin Rodriguez (R.MartinRodriguez@student.tudelft.nl)
* (c) TU Delft, 2023

Make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. Remove `raise NotImplementedError()` afterwards. Moreover, if you see an empty cell, please DO NOT delete it, instead run that cell as you would run all other cells. Please fill in your name(s) and other required details below:

In [ ]:
# Please fill in your names, student numbers, netID, and emails below.
STUDENT_1_NAME = ""
STUDENT_1_STUDENT_NUMBER = ""
STUDENT_1_NETID = ""
STUDENT_1_EMAIL = ""

In [ ]:
# Note: this block is a check that you have filled in the above information.
# It will throw an AssertionError until all fields are filled
assert STUDENT_1_NAME != ""
assert STUDENT_1_STUDENT_NUMBER != ""
assert STUDENT_1_NETID != ""
assert STUDENT_1_EMAIL != ""

### General announcements

* Do *not* share your solutions, and do *not* copy solutions from others. By submitting your solutions, you claim that you alone are responsible for this code.

* Do *not* email questions directly, since we want to provide everybody with the same information and avoid repeating the same answers. Instead, please post your questions regarding this assignment in the correct support forum on Brightspace, this way everybody can benefit from the response. If you do have a particular question that you want to ask directly, please use the scheduled Q&A hours to ask the TA.

* There is a strict deadline for each assignment. Students are responsible to ensure that they have uploaded their work in time. So, please double check that your upload succeeded to the Brightspace and avoid any late penalties.

* This [Jupyter notebook](https://jupyter.org/) uses `nbgrader` to help us with automated tests. `nbgrader` will make various cells in this notebook "uneditable" or "unremovable" and gives them a special id in the cell metadata. This way, when we run our checks, the system will check the existence of the cell ids and verify the number of points and which checks must be run. While there are ways that you can edit the metadata and work around the restrictions to delete or modify these special cells, you should not do that since then our nbgrader backend will not be able to parse your notebook and give you points for the assignment. You are free to add additional cells, but if you find a cell that you cannot modify or remove, please know that this is on purpose.

* This notebook will have in various places a line that throws a `NotImplementedError` exception. These are locations where the assignment requires you to adapt the code! These lines are just there as a reminder for youthat you have not yet adapted that particular piece of code, especially when you execute all the cells. Once your solution code replaced these lines, it should accordingly *not* throw any exceptions anymore.

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

# Task 2a.3 - PD + feedforward control (1.5p)

In this task, you will implement a PD + feedforward controller. The controller consists of the feedforward term

\begin{equation}
\tau_\mathrm{ff} = M(\theta^\mathrm{d}) \: \ddot{\theta}^\mathrm{d} + C(\theta^\mathrm{d}, \dot{\theta}^\mathrm{d}) \: \dot{\theta}^\mathrm{d} + G(\theta^\mathrm{d}),
\end{equation}

and the PD feedback term

\begin{equation}
\tau_\mathrm{fb} = k_\mathrm{p} \, (\theta^\mathrm{d}-\theta) + k_\mathrm{d} \, (\dot{\theta}^\mathrm{d}-\dot{\theta}).
\end{equation}

## Question 1 (0.5p)

Which effects / forces does the term $C(\theta^\mathrm{d}, \dot{\theta}^\mathrm{d}) \: \dot{\theta}^\mathrm{d}$ in the feedforward controller compensate?

**A:** Inertia forces   

**B:** Coriolis & Centrifugal forces   

**C:** Time delays 

**D:** Disturbances

**E:** The joint stiffness

**F:** Friction effects

In [ ]:
# please write the answer ("A", "B", "C", "D", "E", or "F") into the `answer_1` variable
answer_1 = None
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# DO NOT REMOVE OR MODIFY THIS CELL
assert answer_1 in [
    "A",
    "B",
    "C",
    "D",
    "E",
    "F",
], 'Please answer "A", "B", "C", "D", "E", or "F"'


In [ ]:
# Reloads the python files outside of this notebook automatically
%load_ext autoreload
%autoreload 2

# import all Python modules
from distutils.util import strtobool
from functools import partial
from IPython.display import display, HTML  # For animations in the notebook
from jax.config import config as jax_config

jax_config.update("jax_platform_name", "cpu")  # set default device to 'cpu'
jax_config.update("jax_enable_x64", True)  # double precision
from jax import numpy as jnp
import os
from pathlib import Path

from jax_double_pendulum.dynamics import dynamical_matrices
from jax_double_pendulum.robot_parameters import ROBOT_PARAMS

# define boolean to check if the notebook is run for the purposes of autograding
AUTOGRADING = strtobool(os.environ.get("AUTOGRADING", "false"))

## Implementing `ctrl_ff_pure_feedforward` (0.5p)
Please implement the feedforward term of the controller into the function `ctrl_ff_pure_feedforward` in the notebook `controllers.ipynb`. You can use `M, C, G = dynamical_matrices_fn(.)` to access the components of the equations of motion.


In [ ]:
# DO NOT REMOVE OR MODIFY THIS CELL

# import feedforward controller from controllers.ipynb
from ipynb.fs.full.controllers import ctrl_fb_pd, ctrl_ff_feedforward

# construct feedforward controller
ctrl_ff = partial(ctrl_ff_feedforward, partial(dynamical_matrices, ROBOT_PARAMS))

# test output shape of ctr_ff_pure_feedforward
_th, _th_d = jnp.ones((2,)), jnp.ones((2,))
_th_des, _th_d_des, _th_dd_des = jnp.ones((2,)), jnp.ones((2,)), jnp.ones((2,))
_tau_dummy = ctrl_ff(_th, _th_d, _th_des, _th_d_des, _th_dd_des)
print("Computed torque:", _tau_dummy, "Nm")
assert _tau_dummy.shape == (2,), "The computed torque needs to be of shape (2, )"


## Simulating the closed-loop system (0p)

Please simulate the closed-loop system using a PD + pure feedforward controller for regulating the system to follow the given ellipse trajectory. You can use the notebook from Task 2a.1 as an inspiration. Analogue to Task 2a.1, please initialize the double pendulum at `th_0`. This time, you don't need to tune the feedback gains yourself as they are already set to $k_\mathrm{p} = \mathrm{diag}(5000, 5000)$ and $k_\mathrm{d} = \mathrm{diag}(50, 50)$.

In [ ]:
from jax_double_pendulum.analysis import *
from jax_double_pendulum.motion_planning import (
    generate_ellipse_trajectory,
    ELLIPSE_PARAMS,
)
from jax_double_pendulum.robot_simulation import simulate_robot
from jax_double_pendulum.visualization import animate_robot

# simulation parameters
sim_duration = 10.0  # [s]
sim_dt = 1e-2  # [s]

# define time steps
t_ts = sim_dt * jnp.arange(int(sim_duration / sim_dt))

# generate trajectory
traj_ts = generate_ellipse_trajectory(
    rp=ROBOT_PARAMS,
    t_ts=t_ts,
    **ELLIPSE_PARAMS,
)

# initial link angles (i.e. at the first time-step)
th_0 = traj_ts["th_ts"][0] - jnp.array([0.1, 0.2])

# controller parameters
# define the proportional gain matrix kp and the derivative gain matrix kd
kp = 5000 * jnp.eye(2)  # [Nm/rad]
kd = 50 * jnp.eye(2)  # [Nm s/rad]

In [ ]:
# simulate the robot with the given controller
# YOUR CODE HERE
raise NotImplementedError()

# evaluate the control performance quantitatively
rmse_th, rmse_th_d, rmse_th_dd = compute_configuration_space_rmse(traj_ts, sim_ts)
rmse_x, rmse_x_d, rmse_x_dd = compute_operational_space_rmse(traj_ts, sim_ts)
with jnp.printoptions(precision=3):
    print(
        "RMSE theta:",
        rmse_th,
        "rad, RMSE theta_d:",
        rmse_th_d,
        "rad/s, RMSE theta_dd:",
        rmse_th_dd,
        "rad/s^2",
    )
    print(
        "RMSE x:",
        f"{jnp.linalg.norm(rmse_x):.4f}",
        "m, RMSE x_d:",
        f"{jnp.linalg.norm(rmse_x_d):.3f}",
        "m/s, RMSE x_dd:",
        f"{jnp.linalg.norm(rmse_x_dd):.2f}",
        "m/s^2",
    )

In [ ]:
# define folder where to save animations and plots
outputs_dir = Path("outputs")
outputs_dir.mkdir(parents=True, exist_ok=True)

# plot the configuration-space evolution
plot_configuration_space_trajectory_following(
    traj_ts,
    sim_ts,
    filepath=str(
        outputs_dir / "task_2a-3_configuration_space_trajectory_following.pdf"
    ),
)

In [ ]:
# plot the operational-space evolution
plot_operational_space_trajectory_following(
    traj_ts,
    sim_ts,
    filepath=str(outputs_dir / "task_2a-3_operational_space_trajectory_following.pdf"),
)

In [ ]:
# plot the actuation sequence
plot_actuation(sim_ts, filepath=str(outputs_dir / "task_2a-3_actuation.pdf"))

In [ ]:
if not AUTOGRADING:
    ani = animate_robot(
        ROBOT_PARAMS,
        traj_ts=traj_ts,
        sim_ts=sim_ts,
        step_skip=5,
        show=False,
        filepath=str(outputs_dir / "task_2a-3_controlled_robot.mp4"),
    )
    display(HTML(ani.to_html5_video()))

## Removing the feedback controller (0.5p)

Now, set the gains of the PD feedback controller all to zero (i.e. effectively deactivate the feedback controller) and initialize the robot at the first time-step perfectly on the trajectory (instead of offsetting by $(-0.1, -0.2)$ rad, as we usually do). Why is the controller not able to track the trajectory and why does the closed-loop system become unstable?

**A:** Frictional effects are neglected in the control law.

**B:** The time discretization causes the discretized system to drift-off the trajectory defined in the continous time domain.

**C:** As soon as the double pendulum slightly deverts from the trajectory, there exists a mismatch between the dynamics compensated by the controller and the actual system behaviour. This mismatch then causes the system to drift-off even more.

**D:** The stiffness of the joints is not compensated by the controller.

In [ ]:
# please state **all** correct answers ("A", "B", "C", or "D") into a list in the `answer_2` variable.
answer_2 = []
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# DO NOT REMOVE OR MODIFY THIS CELL
assert type(answer_2) == list
for answer_2_element in answer_2:
    assert answer_2_element in ["A", "B", "C", "D"], 'Please answer "A", "B", "C", "D"'
